# Stock Trading Data Preparation

Prepare historical stock prices datasets for demo analysis.

## Original Source

The original data source if on Kaggle: [**https://www.kaggle.com/datasets/ehallmar/daily-historical-stock-prices-1970-2018**](https://www.kaggle.com/datasets/ehallmar/daily-historical-stock-prices-1970-2018)

- This dataset is cleaned up to only include major tech companies (from the list below)
- It is also truncated to only include 2012-2018 data

The resulting file is stored in: [`data/historical_tech_stock_prices.csv`](../../data/historical_tech_stock_prices.csv)

### Download Original Dataset

Download and unzip the original dataset (1.8GB) from Kaggle.

In [2]:
import requests
import zipfile
import os

# URL of the file to be downloaded
url = "https://www.kaggle.com/api/v1/datasets/download/ehallmar/daily-historical-stock-prices-1970-2018"

# Destination path for the downloaded file
data_dir = r"../../data"
zip_file = os.path.join(data_dir, "historical_stocks.zip")
data_file = "historical_stock_prices.csv"  # The specific data file to extract

# Step 1: Download the ZIP file
# Download the file with streaming enabled
print(f"Downloading from kaggle: {url}\nThis will take few minutes...")
response = requests.get(url, stream=True)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Write the file content to the destination file in chunks
    with open(zip_file, "wb") as file:
        print(f"Writing to file: {zip_file}")
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:  # Ensure the chunk is not empty
                file.write(chunk)
    print("Download completed successfully.")
else:
    print(f"Failed to download. Status code: {response.status_code}")
    raise RuntimeError()

# Step 2: Extract only the target file from the ZIP archive
try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Check if the target file exists in the archive
        if data_file in zip_ref.namelist():
            print(f"Extracting zip file: {data_file}")
            zip_ref.extract(data_file, data_dir)
            print(f"Extracted {data_file} to {data_dir}")
        else:
            print(f"Error: {data_file} not found in the archive.")
except zipfile.BadZipFile:
    print("Error: The downloaded file is not a valid ZIP archive.")

# step 3: clean up
print(f"Removing downloaded zip file: {zip_file}")
os.remove(zip_file)


This will take few minutes...
Writing to file: ../../data/historical_stocks.zip
Download completed successfully.
Extracted historical_stock_prices.csv to ../../data


### Clean up Original Dataset

Clean up the data to only include:
- Tech stocks from the list below
- Stocks from 2012-2018

In [14]:
import pandas as pd
import numpy as np
import os

# List of top performing tech companies stock symbols
TECH_SYMBOLS = [
    "AAPL",   # Apple Inc.
    "MSFT",   # Microsoft Corporation
    "GOOGL",  # Alphabet Inc. (Class A)
    "GOOG",   # Alphabet Inc. (Class C)
    "AMZN",   # Amazon.com Inc.
    "FB",     # Meta Platforms, Inc. (formerly Facebook)
    "NFLX",   # Netflix, Inc.
    "NVDA",   # NVIDIA Corporation
    "TSLA",   # Tesla, Inc.
    "INTC",   # Intel Corporation
    "CSCO",   # Cisco Systems, Inc.
    "ADBE",   # Adobe Inc.
    "ORCL",   # Oracle Corporation
    "IBM",    # International Business Machines Corporation
    "CRM",    # Salesforce, Inc.
    "PYPL",   # PayPal Holdings, Inc.
    "AMD",    # Advanced Micro Devices, Inc.
    "TXN",    # Texas Instruments Incorporated
    "QCOM",   # Qualcomm Incorporated
    "AVGO",   # Broadcom Inc.
    "SHOP",   # Shopify Inc.
    "SNAP",   # Snap Inc.
    "TWTR",   # Twitter, Inc.
    "SQ",     # Block, Inc. (formerly Square)
    "DOCU"    # DocuSign, Inc.
]

data_dir = r"../../data"
data_file = os.path.join(data_dir, "historical_stock_prices.csv")
output_file = os.path.join(data_dir, "historical_tech_stock_prices.csv")

# write the headers
with open(output_file, mode='w') as outfile:
    outfile.write("ticker,open,close,adj_close,low,high,volume,date\n")
# load csv
chunks = pd.read_csv(data_file, chunksize=10000)
for df in chunks:
    df = df[(df['ticker'].isin(TECH_SYMBOLS)) & (df['date'] >= '2012-01-01')]
    # sort and output
    df.to_csv(output_file, mode='a', index=False, header=False)

# read back and sort by date and ticker
df = pd.read_csv(output_file)
df.sort_values(by=['date', 'ticker'], ignore_index=True, inplace=True)
df.to_csv(output_file, index=False, header=True)

# print min/max reported stock dates per ticker
df[['ticker', 'date']].groupby(by='ticker').agg(['min', 'max'])

<br/>
<hr/>
<br/>

## Simulate Daily Stock Trades 

Based on the cleaned up tech historical prices (above), generate a series of daily trades performed by different brokers.

- Stocks are traded in **logarithmic** daily distribution where some stocks are traded at higher quantities

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta, datetime

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Generate brokers
broker_names = [
    "Slick Sam", "Trading Tina", "Money Mike", "Clever Cathy", "Profit Pete", 
    "Risky Rachel", "Big Bucks Bob", "Smart Susan", "Lucky Luke"
]
brokers = broker_names

# Parameters
min_trades_per_day, max_trades_per_day = 50, 200
share_prct_range = (0.00001, 0.0001)

# Read historical stock price data
data = pd.read_csv("data/historical_tech_stock_prices.csv", parse_dates=['date'])

# Create a list to hold generated trades
trades = []

# Get min and max dates from the data
min_date = data['date'].min()
max_date = data['date'].max()
# max_date = data['date'].min() + timedelta(days=5)
print(f"generating between: {min_date} - {max_date}")

# Create a date range from min to max
date_range = pd.date_range(start=min_date, end=max_date)

# Traverse the data day by day
for current_date in date_range:
    # Filter data for the current day
    day_data = data[data['date'] == current_date]
    if day_data.empty:
        continue
    
    # Sort tickers for this day and apply a smooth logarithmic curve to prioritize larger trades for certain tickers
    num_tickers = len(day_data['ticker'].unique())
    log_weights = np.logspace(0, -1, num=num_tickers)  # Smooth logarithmic distribution
    day_data = day_data.assign(weight=log_weights)
    day_data = day_data.sort_values(by='weight', ascending=False)
    
    # For each ticker in the current day, generate a series of trades
    for _, row in day_data.iterrows():
        # Get number of trades for the ticker on this day
        num_trades = random.randint(min_trades_per_day, max_trades_per_day)
        
        # Generate trades for the ticker
        for _ in range(num_trades):
            # Random broker
            broker = random.choice(brokers)
            
            # Random timestamps on the current date
            trade_timestamp = current_date + timedelta(seconds=random.randint(0, 86399))
            
            # Bid and ask prices between high and low for the day
            bid_price = np.random.uniform(row['low'], row['high'])
            ask_price = np.random.uniform(row['low'], row['high'])
            while ask_price <= bid_price:  # Ensure ask is higher than bid
                ask_price = np.random.uniform(row['low'], row['high'])
                
            # Trade price between bid and ask
            trade_price = np.random.uniform(bid_price, ask_price)
            
            # Bid-ask spread
            bid_spread = ask_price - bid_price
            
            # Determine number of shares based on the logarithmic weight and the day's volume
            shares = int(row['volume'] * row['weight'] * np.random.uniform(*share_prct_range))
            
            # Calculate trade value
            trade_value = round(trade_price * shares, ndigits=6)
            
            # Create the trade record
            trade = {
                'trade_timestamp': trade_timestamp,
                'ticker': row['ticker'],
                'broker': broker,
                'bid_price': round(bid_price, 4),
                'ask_price': round(ask_price, 4),
                'trade_price': round(trade_price, 4),
                'bid_spread': round(bid_spread, 4),
                'shares': shares,
                'trade_value': round(trade_value, 4),
                # meta columns
                'open': round(row['open'], 6),
                'close': round(row['adj_close'], 6),
                'date': row['date'],
            }
            
            # Include all the original stock data for the day
            # for col in row.index:
            #     trade[col] = row[col]
            
            # Append the trade record to the trades list
            trades.append(trade)
    # finished current date
    print(f"finished date: {current_date}")

# Convert the list of trades into a pandas DataFrame
trades_df = pd.DataFrame(trades)
# sort values
# trades_df.sort_values(by=['trade_timestamp', 'ticker', 'broker'], ignore_index=True, inplace=True)

# Output the generated trades DataFrame to a CSV file
trades_df.to_csv("data/tech_trades.csv", index=False)

print("Trades simulation complete. Output saved to data/tech_trades.csv")



In [5]:
import pandas as pd

# sort trades by timestamp and write back
df = pd.read_csv("data/tech_trades.csv")
df.sort_values(by=['trade_timestamp', 'ticker', 'broker'], ignore_index=True, inplace=True)
# reindex
df.drop(columns=['trade_id'], inplace=True, errors='ignore')
df.insert(0, 'trade_id', df.index + 100000)
df.to_csv("data/tech_trades.csv", index=False)

In [3]:
"""
Rewrite the entire final data file
    - remove teh "Broker #: "
"""
import pandas as pd

# sort trades by timestamp and write back
df = pd.read_csv("data/tech_trades.csv", nrows=10)
df['broker'] = df['broker'].map(lambda x: str(x).split(': ')[1])
df.to_csv("data/tech_trades.csv", index=False)
# display(df)

In [ ]:
import pandas as pd
import numpy as np

# sort trades by timestamp and write back
df = pd.read_csv("data/tech_trades.csv", nrows=10000)
schema = {
    k: 'string' if str(v) == 'O' else \
       'string' if str(v) == 'object' else \
       'float' if str(v) == 'float64' else \
       'integer' if str(v) == 'int64' else \
       str(v)
    for k, v in dict(df.dtypes).items()
}

print(schema)